In [8]:
import backtrader as bt
import alpaca_backtrader_api as api
import alpaca_trade_api as tradeapi
import sys
sys.path.insert(0, '../')
import pandas as pd
import talib as ta
import numpy as np
from config import key_id, secret_key

In [9]:
#import cross signal generator
from cross_signal import *

In [10]:
import random
from datetime import datetime, timedelta


# or a function
def gen_datetime(min_year=1900, max_year=2019):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

In [4]:
#trading data is called in the form of JSON
#https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2019-01-01/2019-02-01?apiKey=


api = tradeapi.REST(key_id, secret_key)

In [5]:
#generate list of random days into list
day_list=[]
day_after_list=[]
for i in range(199):
    rand_day=gen_datetime(2009)
    day_list.append(rand_day.strftime("%Y-%m-%d"))
    day_after=rand_day + timedelta(days=14)
    day_after_list.append(day_after.strftime("%Y-%m-%d"))

In [6]:
day_list[0]

'2016-01-24'

In [7]:
#generate random list of symbols
sym_list=['JACK']
# ,'AAPL','TSLA','JACK'
#for i in range(10):
    #rand_sym=random.random(range=len(symbols.csv))
    #symlist.append(rand_sym)

In [8]:
#Generate random day data for
df_list=[]
for sym in sym_list: 
    data_day_list=[]
    for i in range(len(day_list)):
        data_sym = api.polygon.historic_agg_v2(sym, 1, 'minute', _from=day_list[i], to=day_after_list[i]).df
        data_day_list.append(data_sym)
    df_list.append(data_day_list)

In [7]:
cross_sign=crossed_below(df_list[0][0].high,60)
cross_sign

NameError: name 'df_list' is not defined

In [10]:
for i in range(len(df_list[0])):
    if cross_sign[i]== True:
        print("cross")

cross
cross
cross
cross
cross
cross
cross
cross
cross


# Testing

In [11]:
#feed strat to cerebro

class Scalp(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.fastd < 30 and \
            self.fastk < 30:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastd > 70  and \
            self.rsi > 87  and \
            self.fastk > 70:
                self.sell(size=30)


In [12]:
#feed strat to cerebro

class Scalp4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=5)
        self.emaclose=bt.indicators.EMA(self.data.close, period=5)
        self.emalow=bt.indicators.EMA(self.data.low, period=5)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=5, slowk_period=3, slowd_period=3)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.rsi < 29  and \
            self.fastd < 30 and \
            self.fastk > self.fastd and \
            self.fastk < 30:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastd > 70  and \
            self.rsi > 80  and \
            self.fastk > 70:
                self.sell(size=30)


In [13]:
#feed strat to cerebro

class Scalpy(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        self.mfi=bt.talib.MFI(self.data.high , self.data.low ,self.data.close ,self.data.volume, timeperiod=14 )
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        
        
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.fastk < 30 and \
            self.rsi < 25 and \
            self.cci <= -120.0:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastk > 70 and \
            self.fastd > 70  and \
            self.rsi > 83  and \
            self.cci >= 120.0 :
                self.sell(size=30)


In [14]:
#feed strat to cerebro

class Scalpy3(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 20 and \
            self.fastk < 20 and \
            self.fastd < 20 and \
            self.cci <= -150.0:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastk > 95 and \
            self.fastd > 95  and \
            self.cci >= 150.0 :
                self.sell(size=30)


In [15]:
'''            self.mfi < 30 and \
self.cci <= -100.0 and \
            '''

'            self.mfi < 30 and self.cci <= -100.0 and             '

In [16]:
#feed strat to cerebro

class Scalpy2(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        self.mfi=bt.talib.MFI(self.data.high , self.data.low ,self.data.close ,self.data.volume, timeperiod=14 )
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        
        
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.fastk < 30 and \
            self.fastd < 30 and \
            self.cci <= -130.0:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastk > 75 and \
            self.fastd > 75  and \
            self.rsi > 85  and \
            self.cci >= 100.0 :
                self.sell(size=30)


In [17]:
#feed strat to cerebro

class Scalp2(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.fastd < 30 and \
            self.fastk < 30:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastd > 70  and \
            self.rsi > 90  and \
            self.fastk > 70:
                self.sell(size=30)


In [18]:
# Run over everything
SQN_dict_list=[] 
tradeanalysis_dict_list=[]
ret_dict_list=[]
trans_dict_list=[]
pnl_list=[]

for i in range(len(df_list[0])):
    #create backtrader
    cerebro = bt.Cerebro()

    #cerebro.addstrategy(SmaCross)
    #cerebro.addstrategy(Scalp)

    #Add slippage
    cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005) 

    #set up cerebro

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.0)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=50)

    # Add the analyzers we are interested in
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
    cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")


    # Add the observers we are interested in 
    #this adds more things to the plot on top of the stocks chart
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.BuySell)
    
    
    
    
    cerebro.addstrategy(Scalp)
    datacere = bt.feeds.PandasData(dataname=df_list[0][i])
    cerebro.adddata(datacere,name=day_list[i])
#    print(day_list[i]+' added')
    test=cerebro.run()
    
    
    
    
    
    SQN_dict=test[0].analyzers.sqn.get_analysis() 
    SQN_dict_list.append(SQN_dict)
    
    tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
    tradeanalysis_dict_list.append(tradeanalysis_dict)
    
    ret_dict=test[0].analyzers.ret.get_analysis()
    ret_dict_list.append(ret_dict)
    
    trans_dict=test[0].analyzers.trans.get_analysis()
    trans_dict_list.append(trans_dict_list)
    

In [19]:
SQN_total=0
for i in range(len(SQN_dict_list)):    
    SQN_total=SQN_total+SQN_dict_list[i]['sqn']

SQN_avg=SQN_total/len(SQN_dict_list)

In [20]:
SQN_avg

0.9448794005361731

In [21]:
tradeanalysis_dict_list[0]

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 2), ('open', 1), ('closed', 1)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 1),
                                                     ('longest', 1)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 0)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      21.51900000000026),
                                                     ('average',
                                                      21.51900000000026)])),
                                   ('net',
                                    AutoOrderedDict([('total',
                                      

In [22]:
win_total=0
total_won_pnl=0
avg_won=0

lost_total=0
total_lost_pnl=0
avg_lost=0

total_pnl=0
total_avg=0

for i in range(len(df_list[0])):
    try:
        win_total=win_total+tradeanalysis_dict_list[i]['won']['total']
        total_won_pnl=total_won_pnl+tradeanalysis_dict_list[i]['won']['pnl']['total']
        avg_won=avg_won+tradeanalysis_dict_list[i]['won']['pnl']['average']

        lost_total=lost_total+tradeanalysis_dict_list[i]['lost']['total']
        total_lost_pnl=total_lost_pnl+tradeanalysis_dict_list[i]['lost']['pnl']['total']
        avg_lost=avg_lost+tradeanalysis_dict_list[i]['lost']['pnl']['average']

        total_pnl=total_pnl+tradeanalysis_dict_list[i]['pnl']['gross']['total']
        total_avg=total_avg+tradeanalysis_dict_list[i]['pnl']['gross']['average']
    except:
        pass


In [23]:
win_total , total_won_pnl , avg_won 

(242, 14352.141, 9104.528149999998)

In [24]:
lost_total , total_lost_pnl , avg_lost

(110, -5318.97, -4406.675)

In [25]:
total_pnl , total_avg

(9033.170999999997, 4957.5534499999985)

In [26]:
win_total/(win_total+lost_total)

0.6875

In [27]:
total_pnl/(total_pnl+lost_total)

0.9879691629960765

In [28]:
avg_won/(-avg_lost)

2.0660766110502813

In [29]:
# print the SQN analyzers  SystemQualityNumber
#test[0] is the first strategy used. For multiple strategies, use corresponding number in the list.
#SQN_dict=test[0].analyzers.sqn.get_analysis() 
#tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
#ret_dict=test[0].analyzers.ret.get_analysis()
#trans_dict=test[0].analyzers.trans.get_analysis()


In [30]:
trans_dict

OrderedDict([(datetime.datetime(2014, 3, 26, 16, 20),
              [[30, 77.9607, 0, '2014-03-26', -2338.821]]),
             (datetime.datetime(2014, 3, 28, 9, 42),
              [[-30, 77.1, 0, '2014-03-26', 2313.0]]),
             (datetime.datetime(2014, 3, 28, 11, 20),
              [[30, 77.1071, 0, '2014-03-26', -2313.213]]),
             (datetime.datetime(2014, 3, 28, 20, 4),
              [[-30, 76.7186, 0, '2014-03-26', 2301.558]]),
             (datetime.datetime(2014, 3, 31, 15, 13),
              [[30, 76.7935, 0, '2014-03-26', -2303.805]])])

In [31]:
len(trans_dict)

5